<a href="https://colab.research.google.com/github/bshriselvamkumar/Transcribe/blob/main/LocalTranscribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install faster-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# --- SETTINGS tweak ---
DEFAULT_MAX_WORKERS = 1   # keep 1 for single GPU; bump on CPU or multi-GPU
BEAM_SIZE = 1             # greedy -> fastest
CHUNK_LENGTH = 60         # bigger chunks -> less overhead
USE_VAD = False           # turn off VAD if speed is priority

# --- improved model loader ---
def load_best_model():
    if torch.cuda.is_available():
        logger.info("🔥 GPU FOUND → Loading Whisper (GPU, float16)")
        return WhisperModel(
            "large-v3",               # or "medium" if memory/VRAM limits
            device="cuda",
            compute_type="float16",   # fast on modern GPUs
            local_files_only=False
        )
    else:
        # detect available CPU threads to give to faster_whisper
        cpu_threads = max(1, os.cpu_count() // 2)
        logger.info(f"⚠️ GPU NOT FOUND → Using Whisper (CPU, int8) threads={cpu_threads}")
        return WhisperModel(
            "medium",
            device="cpu",
            compute_type="int8",
            cpu_threads=cpu_threads,
            num_workers=1,
            local_files_only=False
        )

# --- helper: pre-convert audio to 16k mono (optional but recommended) ---
def ensure_audio_16k(in_path, out_path):
    # Requires ffmpeg available in the environment (Colab has ffmpeg)
    if out_path.exists():
        return out_path
    cmd = f'ffmpeg -y -i "{in_path}" -ac 1 -ar 16000 -vn -hide_banner -loglevel error "{out_path}"'
    os.system(cmd)
    return out_path

# --- transcribe (with timing & safer settings) ---
import time
def transcribe_video(filepath, processed, model):
    file_id = filepath.name

    with processed_lock:
        if file_id in processed:
            logger.info(f"⏭️ Already processed (from JSON): {file_id}")
            return {"status": "cached", "file": file_id}

    partial = is_already_transcribed(filepath)
    if partial:
        with processed_lock:
            processed[file_id] = partial.name
            save_processed(processed)
        logger.info(f"⏭️ Found existing transcript file → marking done: {file_id}")
        return {"status": "auto-resumed", "file": file_id}

    try:
        logger.info(f"🎧 Transcribing: {file_id}")

        # optional: fast audio-only conversion path (speeds up repeated ffmpeg work)
        cache_audio = CACHE_DIR / f"{safe_name(filepath.stem)}.wav"
        try:
            ensure_audio_16k(filepath, cache_audio)
            source_for_transcribe = str(cache_audio)
        except Exception:
            # fallback to original file if conversion fails
            source_for_transcribe = str(filepath)

        t0 = time.time()
        segments, info = model.transcribe(
            source_for_transcribe,
            beam_size=BEAM_SIZE,
            vad_filter=USE_VAD,
            language="ta",
            chunk_length=CHUNK_LENGTH,
            word_timestamps=False,   # disable if you don't need word timestamps
            initial_prompt=None
        )
        elapsed = time.time() - t0
        logger.info(f"⏱ Transcription time for {file_id}: {elapsed:.1f}s (audio_len={info.duration:.1f}s)")

        text = " ".join(seg.text for seg in segments)

        out_path = OUT_DIR / f"{safe_name(filepath.stem)}.txt"
        with open(out_path, "w", encoding="utf-8") as f:
            f.write(f"{filepath.name}\n\n{text}")

        with processed_lock:
            processed[file_id] = out_path.name
            save_processed(processed)

        return {"status": "ok", "file": file_id, "time_s": elapsed}

    except Exception as e:
        logger.error(f"❌ Failed: {filepath} | {e}")
        return {"status": "failed", "file": file_id}


In [ ]:
CHUNK_SEC = 1200
CHUNK_LENGTH_WHISPER = 120
MODEL_SIZE_GPU = "medium"
BEAM_SIZE = 1
USE_VAD = False
WORD_TIMESTAMPS = False
USE_CPU_FALLBACK = False


In [ ]:
# High-accuracy Tamil video transcriber (optimized: skip re-encode, smart-splitting, fewer larger chunks)
import os
import re
import json
import time
import math
import logging
import threading
import subprocess
import shutil
import tempfile
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from unidecode import unidecode
from faster_whisper import WhisperModel
from google.colab import files
import torch

# ---------------- CONFIG ----------------
OUT_DIR = Path("/content/t")
CACHE_DIR = Path("/content/cache")
CHUNKS_DIR = Path("/content/chunks")
PROCESSED_JSON = Path("/content/processed.json")
LOG_FILE = Path("/content/transcript.log")

OUT_DIR.mkdir(parents=True, exist_ok=True)
CACHE_DIR.mkdir(parents=True, exist_ok=True)
CHUNKS_DIR.mkdir(parents=True, exist_ok=True)

# Chunk length for splitting source media (seconds).
# 300 = 5 minutes is a safe default; raise/lower as needed.
CHUNK_SEC = 300

# Model / speed knobs
FAST_MODE = True
MODEL_SIZE_GPU = "medium"   # if GPU available and FAST_MODE True
MODEL_SIZE_CPU = "small"    # CPU fallback model
BEAM_SIZE = 1
# internal whisper chunk_length for better perf — tune 30 -> 60 -> 120
CHUNK_LENGTH_WHISPER = 60
USE_VAD = False
WORD_TIMESTAMPS = False
DEFAULT_MAX_WORKERS = 1

# Logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
    handlers=[
        logging.FileHandler(str(LOG_FILE), encoding="utf-8"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("tamil_transcriber")
processed_lock = threading.Lock()

# ---------------- Helpers ----------------
def safe_name(title):
    base = unidecode(title)
    base = re.sub(r"[^A-Za-z0-9_\- ]+", "", base)
    base = base.strip().replace(" ", "_")
    return base[:120] or "untitled"


def load_processed():
    if PROCESSED_JSON.exists():
        try:
            return json.loads(PROCESSED_JSON.read_text(encoding="utf-8"))
        except Exception:
            logger.warning("⚠ processed.json corrupted → resetting")
            return {}
    return {}


def save_processed(data):
    tmp = PROCESSED_JSON.with_suffix(".tmp")
    tmp.write_text(json.dumps(data, indent=2, ensure_ascii=False), encoding="utf-8")
    os.replace(tmp, PROCESSED_JSON)


def ffmpeg_installed():
    return shutil.which("ffmpeg") is not None and shutil.which("ffprobe") is not None


def probe_audio(path: Path):
    """Return (channels:int, samplerate:int, duration:float). Uses ffprobe."""
    if not ffmpeg_installed():
        return 1, 16000, 0.0
    cmd = [
        "ffprobe", "-v", "error",
        "-show_entries", "format=duration",
        "-show_streams",
        "-of", "json", str(path)
    ]
    out = subprocess.check_output(cmd)
    info = json.loads(out)
    # pick the first audio stream if present
    streams = info.get("streams", [])
    channels = 1
    sr = 16000
    if streams:
        stream = streams[0]
        channels = int(stream.get("channels", 1))
        sr = int(stream.get("sample_rate", 16000))
    duration = float(info.get("format", {}).get("duration", 0.0))
    return channels, sr, duration


def split_into_chunks(src_path: Path, out_dir: Path, chunk_sec: int):
    """Split media into fixed-length chunks using ffmpeg.
       Returns list of chunk file paths in order.
       This implementation probes the file and skips re-encoding when possible.
    """
    if not ffmpeg_installed():
        raise RuntimeError("ffmpeg/ffprobe not found in PATH. Install ffmpeg to enable splitting.")
    out_dir.mkdir(parents=True, exist_ok=True)
    safe = safe_name(src_path.stem)

    channels, sr, duration = probe_audio(src_path)
    logger.info(f"probe: channels={channels}, sr={sr}, duration={duration:.1f}s")

    # If file is short enough, don't split — return original path to be transcribed directly.
    if duration <= chunk_sec:
        logger.info("File shorter than chunk size — skipping split")
        return [src_path]

    # Calculate number of chunks and segment_time to avoid many tiny chunks
    num_chunks = math.ceil(duration / chunk_sec)
    # segment_time will be <= chunk_sec, distributed evenly
    segment_time = int(math.ceil(duration / num_chunks))
    # ensure a reasonable minimum (avoid many tiny segments for tiny duration errors)
    segment_time = max(30, segment_time)

    pattern = out_dir / f"{safe}_chunk_%04d.wav"

    # If already mono 16000 we can copy codec to avoid costly re-encode
    if channels == 1 and sr == 16000:
        cmd = [
            "ffmpeg", "-y", "-i", str(src_path),
            "-f", "segment", "-segment_time", str(segment_time),
            "-reset_timestamps", "1",
            "-acodec", "copy",
            str(pattern)
        ]
    else:
        cmd = [
            "ffmpeg", "-y", "-i", str(src_path),
            "-vn", "-ac", "1", "-ar", "16000",
            "-f", "segment", "-segment_time", str(segment_time),
            "-reset_timestamps", "1",
            str(pattern)
        ]
    logger.info("Splitting with ffmpeg: " + " ".join(cmd))
    subprocess.check_call(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    chunks = sorted(out_dir.glob(f"{safe}_chunk_*.wav"))
    logger.info(f"Produced {len(chunks)} chunks (segment_time={segment_time}s)")
    return chunks


def load_best_model(use_gpu=True, local_files_only=False):
    try:
        if torch.cuda.is_available() and use_gpu:
            model_name = MODEL_SIZE_GPU if FAST_MODE else "large-v3"
            logger.info(f"🔥 GPU FOUND → Loading Whisper {model_name} (GPU, float16)")
            return WhisperModel(model_name, device="cuda", compute_type="float16", local_files_only=local_files_only)
        else:
            model_name = MODEL_SIZE_CPU if FAST_MODE else "medium"
            cpu_threads = max(1, (os.cpu_count() or 1) // 2)
            logger.info(f"⚠️ GPU NOT FOUND / using CPU → Whisper {model_name} (int8), threads={cpu_threads}")
            return WhisperModel(model_name, device="cpu", compute_type="int8", cpu_threads=cpu_threads, num_workers=1, local_files_only=local_files_only)
    except Exception as e:
        logger.exception("Failed to load model: %s", e)
        raise


def is_already_transcribed(filepath: Path):
    expected = OUT_DIR / f"{safe_name(filepath.stem)}.txt"
    if expected.exists() and expected.stat().st_size > 20:
        return expected
    return None

# ---------------- Transcription logic (chunked, incremental) ----------------
def transcribe_file_chunked(src_path: Path, model, processed):
    file_id = src_path.name
    safe = safe_name(src_path.stem)
    out_path = OUT_DIR / f"{safe}.txt"

    # If marked done in JSON
    with processed_lock:
        entry = processed.get(file_id, {})

    if entry.get("status") == "done" and out_path.exists():
        logger.info(f"⏭ Already completed (JSON): {file_id}")
        return {"status": "cached", "file": file_id}

    # If we have previously split chunks saved in processed.json, reuse them instead of re-splitting
    manifest = entry.get("chunks_manifest")
    if manifest:
        chunks = [Path(p) for p in manifest]
        # verify chunks still exist
        if not all(p.exists() for p in chunks):
            logger.info("Chunk manifest exists but files missing — re-splitting")
            chunks = split_into_chunks(src_path, CHUNKS_DIR, CHUNK_SEC)
    else:
        # fresh split (or skip splitting if file is short)
        try:
            chunks = split_into_chunks(src_path, CHUNKS_DIR, CHUNK_SEC)
        except Exception as e:
            logger.exception("Failed to split file: %s", e)
            return {"status": "failed_split", "file": file_id, "error": str(e)}

    if not chunks:
        logger.error("No chunks produced for %s", src_path)
        return {"status": "no_chunks", "file": file_id}

    # Save manifest to processed.json so future runs can resume without re-splitting
    with processed_lock:
        processed[file_id] = processed.get(file_id, {})
        processed[file_id]["chunks_manifest"] = [str(p) for p in chunks]
        save_processed(processed)

    # Find start index from processed entry
    start_idx = int(entry.get("last_chunk", 0))
    logger.info(f"Transcribing {file_id} → {len(chunks)} chunks, starting at {start_idx}")

    # Open output file in append mode so we save incrementally
    with open(out_path, "a", encoding="utf-8") as out_f:
        # If this is a fresh file (size small) write header
        if out_path.stat().st_size == 0:
            out_f.write(f"{src_path.name}\n\n")

        for idx in range(start_idx, len(chunks)):
            chunk_path = chunks[idx]
            logger.info(f"🎧 Transcribing chunk {idx+1}/{len(chunks)}: {chunk_path.name}")
            t0 = time.time()
            try:
                segments, info = model.transcribe(
                    str(chunk_path),
                    beam_size=BEAM_SIZE,
                    vad_filter=USE_VAD,
                    language="ta",
                    chunk_length=CHUNK_LENGTH_WHISPER,
                    word_timestamps=WORD_TIMESTAMPS
                )
                chunk_text = "\n".join(seg.text.strip() for seg in segments if seg.text.strip())
                # write chunk marker + text
                out_f.write(f"\n\n--- CHUNK {idx+1} / {len(chunks)} ---\n\n")
                out_f.write(chunk_text + "\n")
                out_f.flush()
                elapsed = time.time() - t0
                dur = getattr(info, "duration", 0)
                logger.info(f"✅ Chunk {idx+1} done in {elapsed:.1f}s (audio ≈ {dur:.1f}s)")

                # update processed JSON after each chunk
                with processed_lock:
                    processed[file_id] = {
                        "last_chunk": idx + 1,
                        "chunks_total": len(chunks),
                        "out_file": out_path.name,
                        "status": "in_progress",
                        "chunks_manifest": [str(p) for p in chunks]
                    }
                    save_processed(processed)

            except RuntimeError as e:
                # potential CUDA OOM — try fallback to CPU
                logger.exception("RuntimeError during chunk transcription (attempt fallback): %s", e)
                try:
                    logger.info("Retrying this chunk on CPU model...")
                    cpu_model = load_best_model(use_gpu=False)
                    segments, info = cpu_model.transcribe(
                        str(chunk_path),
                        beam_size=BEAM_SIZE,
                        vad_filter=USE_VAD,
                        language="ta",
                        chunk_length=max(10, CHUNK_LENGTH_WHISPER // 2),
                        word_timestamps=WORD_TIMESTAMPS
                    )
                    chunk_text = "\n".join(seg.text.strip() for seg in segments if seg.text.strip())
                    out_f.write(f"\n\n--- CHUNK {idx+1} / {len(chunks)} (CPU retry) ---\n\n")
                    out_f.write(chunk_text + "\n")
                    out_f.flush()
                    with processed_lock:
                        processed[file_id] = {
                            "last_chunk": idx + 1,
                            "chunks_total": len(chunks),
                            "out_file": out_path.name,
                            "status": "in_progress",
                            "chunks_manifest": [str(p) for p in chunks]
                        }
                        save_processed(processed)
                    logger.info("✅ CPU retry succeeded for chunk %d", idx+1)
                except Exception as e2:
                    logger.exception("CPU retry also failed for chunk %d: %s", idx+1, e2)
                    return {"status": "failed_chunk", "file": file_id, "chunk": idx+1, "error": str(e2)}

            except Exception as e:
                logger.exception("Unexpected error transcribing chunk %d: %s", idx+1, e)
                return {"status": "failed_chunk", "file": file_id, "chunk": idx+1, "error": str(e)}

    # all chunks completed
    with processed_lock:
        processed[file_id] = {"status": "done", "out_file": out_path.name, "chunks_total": len(chunks), "chunks_manifest": [str(p) for p in chunks]}
        save_processed(processed)

    logger.info(f"🎉 File done: {file_id} -> {out_path}")
    return {"status": "ok", "file": file_id, "out_file": out_path.name}


# ---------------- RUNNER ----------------
def run_local_batched(files_uploaded):
    model = load_best_model()
    processed = load_processed()

    # Save uploaded bytes to cache dir
    file_paths = []
    for name, content in files_uploaded.items():
        p = CACHE_DIR / name
        with open(p, "wb") as f:
            f.write(content)
        file_paths.append(p)

    logger.info(f"📁 Total uploaded: {len(file_paths)}")

    # iterate files sequentially (single GPU preferred)
    for p in file_paths:
        already = is_already_transcribed(p)
        if already:
            with processed_lock:
                processed[p.name] = {"status": "done", "out_file": already.name}
                save_processed(processed)
            logger.info(f"⏭ Existing transcript found for {p.name}: {already}")
            continue

        res = transcribe_file_chunked(p, model, processed)
        logger.info("Result: %s", res)

    print("\n🎉 ALL DONE!")
    print(f"📦 Transcripts saved at: {OUT_DIR}")
    print("⏮ Re-run the cell to auto-resume any incomplete files.")


# ---------------- RUN ----------------
print("📂 Upload video/audio files (Colab prompt):")
uploaded = files.upload()
print("\n🚀 Starting chunked transcription...")
run_local_batched(uploaded)


📂 Upload video/audio files (Colab prompt):


Saving ஜாதக ஆய்வு 6, 1ம் வீடு முதல் 12ம் வீடுவரை பலன்கள் step by ste.mp4 to ஜாதக ஆய்வு 6, 1ம் வீடு முதல் 12ம் வீடுவரை பலன்கள் step by ste.mp4

🚀 Starting chunked transcription...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocabulary.txt: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]